<div style="text-align:right"><i>Peter Norvig<br>Jan 2016<br>revised 2018, 2020, 2021</i></div>

# Making Numbers: Countdowns, Four 4s, Five 5s, ...

In this notebook we solve a range of related puzzles that all involve making mathematical expressions by combining numbers and operators in various ways to make target numeric values.  First some imports, and then we can look at the first problem.

In [1]:
from collections import Counter, defaultdict, namedtuple
from fractions   import Fraction
from functools   import lru_cache
from itertools   import product, permutations
from math        import sqrt, factorial, floor, ceil
from operator    import add, sub, mul, neg, truediv as div
from typing      import List, Tuple, Dict, Union, Sequence, Collection, Set, Optional
import re
import functools

cache = functools.cache if 'cache' in functools.__dict__ else functools.lru_cache(None)

# Countdown to 2016

On January 1, 2016 Alex Bellos [posed](http://www.theguardian.com/science/2016/jan/04/can-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) (and subsequently [answered](http://www.theguardian.com/science/2016/jan/04/did-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016)) this New Year's puzzle:

- *Fill in the blanks so that this equation makes arithmetical sense:*

            10 ␣ 9 ␣ 8 ␣ 7 ␣ 6 ␣ 5 ␣ 4 ␣ 3 ␣ 2 ␣ 1 = 2016

- *You are allowed to use *only* the four basic arithmetical operations: +, -, &times;, ÷. But brackets (parentheses) can be used wherever needed. So, for example, the solution could begin as either:*

            (10 + 9) × (8 ...
            10 + (9 × 8) ...

To solve this specific problem, I'll first solve the  more general problem: 

- *Given any sequence of numbers, place operations and brackets to form a dict of `{value: expression}` for every value that can be made.*

I'll define  `expressions(numbers)` to return an **expression table**: a dict of `{value: expression}` for all expressions (strings) whose numeric value is a `value` that can be made using all the `numbers`, for example:

    expressions((10,)) ⇒ {10: '10'}
    expressions((9, 8)) ⇒ {1: '(9-8)', 1.125: '(9/8)', 17: '(9+8)', 72: '(9*8)'}

I'll use the idea of [**dynamic programming**](https://en.wikipedia.org/wiki/Dynamic_programming): break the problem down into simpler subparts, compute an answer for each subpart, and remember intermediate results (with `lru_cache`) so we don't need to re-compute them later. How do we break the problem into parts? If there is only one number, then there is only one expression, the number itself. If there are multiple numbers, then consider all ways of splitting the numbers into two parts, finding all the expressions that can be made with each part, and combining pairs of expressions with any of the four operators (taking care not to divide by 0):

In [2]:
Exp = str # An expression is a string, like '(10+(9-8))'
ExpTable = Dict[float, Exp] # A table, like {11: '(10+(9-8))', ...}
    
@lru_cache(None)
def expressions(numbers: tuple) -> ExpTable:
    """Return a {value: exp} table for all expressions that can be made from `numbers`."""
    if len(numbers) == 1: # expressions((10,)) => {10: '10'}
        return {numbers[0]: str(numbers[0])}
    else: 
        table = {}
        for (Lnums, Rnums) in splits(numbers):
            for (L, R) in product(expressions(Lnums), expressions(Rnums)):
                Lexp, Rexp = '(' + expressions(Lnums)[L], expressions(Rnums)[R] + ')'
                if R != 0: 
                    table[L / R] = Lexp + '/' + Rexp
                table[L * R] = Lexp + '*' + Rexp
                table[L - R] = Lexp + '-' + Rexp
                table[L + R] = Lexp + '+' + Rexp
        return table
    
def splits(sequence, start=1) -> List[Tuple[Sequence, Sequence]]:
    "Split sequence into two non-empty parts, in all ways."
    return [(sequence[:i], sequence[i:]) 
            for i in range(start, len(sequence))]

Some tests to make sure we got this right:

In [3]:
assert splits((3, 2, 1)) == [((3,), (2, 1)), 
                             ((3, 2), (1,))]

assert expressions((3,)) == {3: '3'}
assert expressions((2, 1)) == {1: '(2-1)', 2: '(2*1)', 3: '(2+1)'}
assert expressions((3, 2)) == {1.5: '(3/2)', 6: '(3*2)',  1: '(3-2)', 5: '(3+2)'}
assert expressions((1,)) == {1: '1'}

assert expressions((3, 2, 1)) == {0: '((3-2)-1)', 0.5: '((3/2)-1)', 1: '((3-2)*1)',
 1.5: '((3/2)*1)', 2: '((3-2)+1)', 2.5: '((3/2)+1)', 3: '(3*(2-1))',
 4: '((3+2)-1)', 5: '((3+2)*1)', 6: '((3+2)+1)', 7: '((3*2)+1)', 9: '(3*(2+1))'}

assert splits((5, 4, 3, 2, 1)) == [
    ((5,), (4, 3, 2, 1)),                  
    ((5, 4), (3, 2, 1)),             
    ((5, 4, 3), (2, 1)),         
    ((5, 4, 3, 2), (1,))]

That looks reasonable. Let's solve the whole puzzle.



In [4]:
c10 = (10, 9, 8, 7, 6, 5, 4, 3, 2, 1) # A countdown from 10 to 1

%time expressions(c10)[2016]

CPU times: user 19.1 s, sys: 427 ms, total: 19.5 s
Wall time: 19.5 s


'(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)+1)'

We have an answer! And in just seconds, thanks to dynamic programming! Here are solutions for nearby years:

In [5]:
{y: expressions(c10)[y] for y in range(2010, 2030)}

{2010: '((((((10+(((9*8)-7)*6))*5)+4)+3)+2)+1)',
 2011: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)-1)',
 2012: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)*1)',
 2013: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)+1)',
 2014: '(((((((10+((9*8)*7))-6)-5)*4)+3)-2)+1)',
 2015: '(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)*1)',
 2016: '(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)+1)',
 2017: '(((10*(((9*8)-((7*6)/(5+4)))*3))-2)-1)',
 2018: '(((10*(((9*8)-((7*6)/(5+4)))*3))-2)*1)',
 2019: '(((10*(((9*8)-((7*6)/(5+4)))*3))-2)+1)',
 2020: '(((((10+((9+((8+7)*6))*5))*4)+3)-2)-1)',
 2021: '((((((((10-9)+(8*7))*6)-5)*4)*3)/2)-1)',
 2022: '((((((((10-9)+(8*7))*6)-5)*4)*3)/2)*1)',
 2023: '(((((10*(((9*8)*(7-6))-5))+4)*3)+2)-1)',
 2024: '(((((10*(((9*8)*(7-6))-5))+4)*3)+2)*1)',
 2025: '(((((10*(((9*8)*(7-6))-5))+4)*3)+2)+1)',
 2026: '((((10+((((9*8)*7)*(6-5))*4))+3)-2)-1)',
 2027: '((((10+((((9*8)*7)*(6-5))*4))+3)-2)*1)',
 2028: '((((10+((((9*8)*7)*(6-5))*4))+3)-2)+1)',
 2029: '(((((((10*9)+8)*7)*6)-((5*4)*3))/2)+1)'}

# Counting Countdown Solutions

Alex Bellos had another challenge:  

- *I was half hoping a computer scientist would let me know exactly how many distinct solutions there are with only the four basic operations. Maybe someone will.*

As it stands, my program can't answer that question, because I only keep one expression for each value. 

Also, I'm not sure what it means to be a distinct solution. For example, are `((10+9)+8)` and `(10+(9+8))` different, or are they same, because they both are equivalent to `(10+9+8)`? Similarly, are `((3-2)-1)` and `(3-(2+1))` different, or the same because they both are equivalent to `(3 + -2 + -1)`? I think the notion of "distinct solution" is just inherently ambiguous. My choice is to count each of these as distinct: every expression has exactly ten numbers, nine operators, and nine pairs of brackets, and if an expression differs in any character, it is different. But I won't argue with anyone who prefers a different definition of "distinct solution."

So how can I count expressions? I can mimic `expressions`, but make a table of counts, rather than expression strings. There is only 1 way to make a single number into an expression, and in general, combining two subexpressions into a larger expression using an operator can be done in a number of ways equal to the product of the ways each of the two subexpressions can be made:

In [6]:
@lru_cache(None)
def expression_counts(numbers: tuple) -> Counter:
    "Return a Counter of {value: count} for every value that can be made from numbers."
    if len(numbers) == 1: # Only one way to make an expression out of a single number
        return Counter(numbers)
    else: 
        table = Counter()
        for (Lnums, Rnums) in splits(numbers):
            for L, R in product(expression_counts(Lnums), expression_counts(Rnums)):
                count = expression_counts(Lnums)[L] * expression_counts(Rnums)[R]
                if R != 0:
                    table[L / R] += count
                table[L + R] += count
                table[L - R] += count
                table[L * R] += count
        return table

In [7]:
assert expression_counts((3,)) == Counter({3: 1})
assert expression_counts((3, 2, 1))[1] == 5 # (3-2)*1, (3-2)/1, 3-(2*1), 3-(2/1), 3/(2+1)

Looks good to me. Now let's see if we can answer Alex's question.

In [8]:
expression_counts(c10)[2016]

30066

This says there are 30,066 distinct expressions for 2016. Is that the answer Alex wanted?

# The Trouble with Round-off Error

I don't think it is the right answer. The trouble is: round-off error.

Let's find all the values in `expressions(c10)` that are very near to 2016, within ±0.0000000001:

In [9]:
{y: expressions(c10)[y]
 for y in expressions(c10)
 if abs(y - 2016) < 1e-10}

{2016.0000000000002: '((((10*((9*(8+7))-(6/(5+4))))*3)/2)+1)',
 2016.0: '(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)+1)',
 2015.9999999999995: '((((((10*9)*8)*7)*((6/5)-(4-3)))*2)*1)',
 2015.999999999997: '(((10*9)*8)/(7-(6-(((5/(4+3))/2)-1))))',
 2016.0000000000005: '(((((10*9)*8)*((((7*6)/5)-4)-3))*2)*1)',
 2016.0000000000018: '((((((10*9)*8)*7)*(((6/5)-4)+3))*2)*1)',
 2016.0000000000023: '(10*((9*8)/((7-(6-((5/(4+3))/2)))-1)))',
 2015.9999999999998: '((10*((9*(((8-(7/6))*5)-(4*3)))+2))+1)',
 2015.9999999999993: '(10*(((((9*8)*7)*6)/5)*(((4/3)-2)+1)))',
 2016.000000000002: '(((10*9)*8)/((7-(6-((5/(4+3))/2)))-1))',
 2015.999999999999: '((((10*9)*(8+7))-6)/(((5-(4/3))-2)-1))'}

I suspect that all of these actually should be *exactly* equal to 2016. To determine if they are, I could re-do *all* the calculations using exact rational arithmetic, as provided by the `fractions.Fraction` class. From experience I know that would be at least an order of magnitude slower, so instead I'll just verify the small set of expressions in the output above. I'll define `exact(exp)` to return a string that, when passed to `eval`, will exactly calculate `exp` using rational arithmetic:

In [10]:
def exact(exp) -> Exp: return re.sub(r"([0-9]+)", r"Fraction(\1)", exp)

exact('1/(5-2)')

'Fraction(1)/(Fraction(5)-Fraction(2))'

In [11]:
assert eval(exact('1/(5-2)')) == Fraction(1, 3)

Now I can count up all the expressions in the `expression_counts(c10)` table that are near 2016, but with `exact` computation to check that the expressions evaluate to exactly 2016:

In [12]:
sum(expression_counts(c10)[y] 
    for y, exp in expressions(c10).items()
    if abs(y - 2016) < 1e-6 and eval(exact(exp)) == 2016)

44499

I can claim that the answer is  **44,499**, but I would feel more confident if I did *all* the computations with exact arithmetic, and if the result was independently verified by someone else, and passed an extensive test suite. And of course, if you have a different definition of "distinct solution," you will get a different answer.

# Four 4s

Alex Bellos continued his original puzzle column with a related puzzle:
    
> The most famous “fill in the gaps in the equation” puzzle is known as  [**four fours**](https://en.wikipedia.org/wiki/Four_fours), because every equation is of the form
>
>           4 ␣ 4 ␣ 4 ␣ 4 = x
>
> In the classic form of the puzzle you must find a solution for x = 0 to 9 using just addition, subtraction, multiplication and division (and brackets).

This puzzle goes back to a [1914 publication](https://archive.org/details/mathematicalrecr00ball) by the mathematician/magician [W. W. Rouse Ball](https://en.wikipedia.org/wiki/W._W._Rouse_Ball). The solution is easy:

In [13]:
{i: expressions((4, 4, 4, 4))[i] for i in range(10)}

{0: '(((4-4)-4)+4)',
 1: '(((4/4)-4)+4)',
 2: '((4/(4+4))*4)',
 3: '(((4+4)+4)/4)',
 4: '(((4-4)*4)+4)',
 5: '(((4*4)+4)/4)',
 6: '(((4+4)/4)+4)',
 7: '((4-(4/4))+4)',
 8: '(((4+4)/4)*4)',
 9: '(((4/4)+4)+4)'}

Note that I didn't do anything special to take advantage of the fact that in `(4, 4, 4, 4)` the digits are all the same. Happily, `@cache` does that for me automatically! If I split that into `(4, 4)` and `(4, 4)`, when it comes time to do the second half of the split, the result will already be in the cache, and so won't be recomputed.

# New Math Operations, and Permutations

Bellos then writes:
    
- *If you want to show off, you can introduce **new mathematical operations** such as powers, square roots, concatenation and decimals, ... or use the factorial symbol, `!`.*

It seems there are a lot of similar puzzles that all have slightly different rules for which numbers are required and which operators are allowed. Many of the puzzles allow **permutations** of the digits: instead of requiring (2, 0, 1, 6) to appear in that order, they are allowed to appear in any order. Some allow **concatenation** of digits (e.g. the digits (1, 6) can form the expression `'16'`) and some allow **decimal points** (e.g., `'1.6'`). Some allow **unary operators** (e.g. `'√9'` or `'-(2*3)'`).

Hanling these additional kinds of expressions introduces some issues; here's how I'll handle them:

- **Imaginaries**: `√-1` is an imaginary number (I won't allow imaginary numbers).
- **Irrationals**: `√2` is an irrational number (I'll use floating point approximations).
- **Round-off error**:`49*(1/49) == 0.9999999999999999`, not `1` (I'll try to round off, but nmight make errors of ommission or commission).
- **Overflow**: `10.^(9.^8.)` gives an `OverflowError` (I'll drop overflow results, possibly making an error of ommission).
- **Unlikely numbers**: `(.9^(4!))^(√2)` is a valid number, but unlikely to eventually lead to an integer. (I'll drop unlikely operations.)
- **Arbitraily deeply nested unary operators**: e.g, `√√√√√√√√(4!!!!!!!!)` (I'll limit the unary operator nesting level to 2).



To facilitate solving a range of puzzles with different operators, I will redefine `expressions` to take a second argument, `ops`, specifying the allowable operators as a string of one-character codes. The table below lists binary operators in the left column; unary operators in the middle; and two pseudo-operations on digits on the right (I chose comma for concatenation, because it is familiar in numbers like: `'1,000,000'`).

|Code|Operator|Code|Operator|Code|Operator|
|------|--------|------|--------|----|---|
|`+`|addition: 1 + 2 = 3      |`_`|unary minus: -2 = -2|`.`|decimal point: 1.23|
|`-`|subtraction: 3 - 2 = 1   |`√`|square root: √9 = 3|`,`|concatenation of digits: 123|
|`*`|multiplication: 2 * 3 = 6|`!`|factorial: 4! = 24 |
|`/`|division: 6 / 3 = 2      |`⌊`|floor:⌊4.4⌋ = 4|
|`^`|exponentiation: 2 ^ 3 = 8|`⌈`|ceiling: ⌈4.4⌉ = 5|


I will define the data type `Operator` to define an operator, giving its code symbol, its arity (binary or unary), the Python function to call to do the calculation, the format string for unary operations, and for some operations, a `guard` that says when it is applicable. For example:
  - Division is applicable when the second argument is not zero.
  - I only allow square roots for a small range of positive numbers that become integers when multiplied by 9!.
  - I only allow factorials for 0 to 9.

These some that are not mathematically necessary, but keep the table from being overpopulated with numbers that are unlikely to help lead to solutions. Unfortunately, that means some good solutions will be missed. 

For binary operations, the symbol is used to construct the expression string, but for unary ops the `fmt` field gives a format string; this allows us to have prefix and postfix operators. The function `operators` picks out the operators you want from the `OPERATORS` global variable. You can augment this with new operators as you wish.

In [14]:
def true(*args): return True

Operator = namedtuple('Operator', 'symbol, func, fmt, guard', defaults=[None, true])

OPERATORS = {
    2: {Operator('+', add),
        Operator('-', sub),
        Operator('*', mul),
        Operator('/', div,       None,  lambda L, R: R != 0),
        Operator('^', pow,       None,  lambda L, R: -10 <= R <= 10 and (L > 0 or R == int(R)))},
    1: {Operator('√', sqrt,      '√{}', lambda v: 0 < v <= 256 and (362880. * v).is_integer()),
        Operator('!', factorial, '{}!', lambda v: v in range(10)),
        Operator('_', neg,       '-{}'),
        Operator('⌊', floor,     '⌊{}⌋'),
        Operator('⌈', ceil,      '⌈{}⌉')}}

OPS = '+-*/^_√!.,' # Default set of operators; omits floor and ceiling.

def operators(arity: int, ops: str) -> List[Operator]:
    """All the operators in OPERATORS with given arity whose code symbol is one of `ops`."""
    return [op for op in OPERATORS[arity] if op.symbol in ops]

I'll define the function `operate` to compute an arithmetic operation, catch any errors, and try to correct round-off errors. The idea is that since my expressions start with integers, results that are close to an integer probably are that integer. So I'll correct `(49*(1/49))` to be `1.0`. Of course, an expression like `(1+(10^-99))` is also very close to `1.0`, but it should not be equal to `1`. Instead, I'll try to avoid such expressions by silently dropping (returning `None` from `operate`)  any intermediate value whose magnitude is outside the range of 10<sup>-10</sup> to 10<sup>10</sup> (except of course I will accept an exact 0).

In [15]:
def operate(operator, *args) -> Optional[float]: 
    "Return op(*args), trying to correct for roundoff error, or `None` if too big/small/erroneous."
    if operator.guard(*args):
        try:
            val = operator.func(*args)
        except (ArithmeticError, ValueError):
            return None
        return (0.0        if val == 0                      else
                None       if isinstance(val, complex)      else
                None       if not (1e-10 < abs(val) < 1e10) else
                round(val) if abs(val - round(val)) < 1e-12 else
                val)

# Refactoring `expressions`

I'll take this opportunity to refactor `expressions` to use the new `OPERATORS`. I introduce  three subfunctions:
- `digit_expressions(digits, ops)`: returns a table of expressions made with all the digits, in the given order.
  - If `','` is in `ops` allow concatenation of digits (e.g. `{44: '44'}`).
  - If `'.'` is in ops allow decimals (e.g. `{4.4: '4.4', 0.44: '.44'}`. 
  - If `digits` is a single digit, always allow it (e.g. `{4: '4'}`).
- `add_binary_expressions(table, numbers, ops, nesting)`: builds binary expressions using all numbers in order (e.g. `{5: '(2+3)'}`).
- `add_unary_expressions(table, ops, nesting)`: add expressions like `√4` and `4!` to `table` and return `table`. 



In [16]:
@cache
def expressions(numbers: Collection[int], ops=OPS, nesting=2, permute=False) -> ExpTable:
    "Return {value: expr} for all expressions that can be made from numbers using ops."
    orderings = (permutations(numbers) if permute else [numbers])
    table = {}
    for nums in orderings:
        table.update(digit_expressions(nums, ops))
        add_binary_expressions(table, nums, ops, nesting)
    return add_unary_expressions(table, ops, nesting)

@cache
def digit_expressions(digits: Tuple[int], ops: str) -> ExpTable:
    """Return {value: expr} for expressions made with all the digits, in the given order, 
    maybe with a decimal point."""
    D = ''.join(map(str, digits))
    table = {}
    if len(digits) == 1 or (',' in ops and not D.startswith('0')): 
        table[int(D)] = D
    if '.' in ops: 
        for L, R in splits(D, 0):
            if len(L) <= 1 or not D.startswith('0'):
                decimal = L + '.' + R
                table[float(decimal)] = decimal
    return table


def add_binary_expressions(table, numbers, ops, nesting) -> ExpTable:
    "Add binary expressions by splitting numbers and combining with an op."
    binary_ops = operators(2, ops)
    for (Lnums, Rnums) in splits(numbers):
        Ltable = expressions(Lnums, ops, nesting, False)
        Rtable = expressions(Rnums, ops, nesting, False)
        for (L, R) in product(Ltable, Rtable):
            Lexp, Rexp = '(' + Ltable[L], Rtable[R] + ')'
            for op in binary_ops:
                assign(table, operate(op, L, R), Lexp + op.symbol + Rexp)
    return table
                
def add_unary_expressions(table, ops: str, nesting: int) -> ExpTable:
    "Add unary expressions (e.g. -v, √v and v!) to table"
    unary_ops = operators(1, ops)
    for _ in range(nesting):
        for v in tuple(table):
            for op in unary_ops:
                assign(table, operate(op, v), op.fmt.format(table[v]))
    return table

The function `assign` adds a `{val: exp}` entry to `table`, but if there is already an entry for `val`, it prefers the entry with the lowest total *weight*: the number of characters plus extra points for particularly complex characters. The idea is to prefer simpler expressions. If you prefer complex expressions, you can change the weights to be negative. The weights don't change how many different values can be made, they just change which of two or more expressions are chosen to represent a value.

In [17]:
def assign(table: dict, val: float, exp: str): 
    "Assign table[val] = exp, unless we already have a lighter exp or val is None."
    if val is not None and (val not in table or weight(exp) < weight(table[val])): 
        table[val] = exp
        
WEIGHTS = Counter({'√':4, '!':2, '.':1, '^':1, '/':0.2, '-':0.1, '⌊': 5, '⌈':5})
        
def weight(exp: str) -> int: return len(exp) + sum(WEIGHTS[c] for c in exp)

In [18]:
add_unary_expressions({16: '16'}, '_', 2) 

{16: '16', -16: '-16'}

That's a lot of new code; let's have some tests:

In [19]:
assert digit_expressions((1, 2), OPS) == {12: '12', 0.12: '.12', 1.2: '1.2'}
assert digit_expressions((1, 2), ',') == {12: '12'}
assert digit_expressions((1, 2), '.') == {.12: '.12', 1.2: '1.2'}
assert digit_expressions((1, 2), '') == {} 
assert digit_expressions((0, 1, 2), OPS) == {0.012: '.012', 0.12: '0.12'}
assert digit_expressions((0, 0, 7), '.') == {0.007: '.007', 0.07: '0.07'}

assert add_unary_expressions({16: '16'}, '_', 2) == {16: '16', -16: '-16'}
assert add_unary_expressions({16: '16'}, OPS, 2) == {16: '16', -16: '-16', 4: '√16', -4: '-√16', 
                                                      2: '√√16', 24: '√16!'}

assert expressions((3, 2), '+-*,') == {32.0: '32', 5: '(3+2)', 1: '(3-2)', 6: '(3*2)'}
assert expressions((1, 2, 3), '+^') == {6:'(1+(2+3))', 1:'(1^(2+3))', 9:'(1+(2^3))', 
                                        27:'((1+2)^3)', 4:'((1^2)+3)'}
assert expressions((1, 2, 3), '+^', permute=True) == {
    1: '(1^(2+3))', 2: '(2^(1^3))', 3: '((1^3)+2)', 4: '((1^2)+3)', 5: '((2^1)+3)', 
    6: '(1+(2+3))', 8: '((2^1)^3)', 9: '(1+(2^3))', 10: '(1+(3^2))', 16: '((1+3)^2)', 27: '((1+2)^3)'}

I'll define a function to print a table of consecutive integers (starting at 0) that can be made by a sequence of numbers:

In [20]:
def show(numbers: tuple, limit=None, ops=OPS, nesting=2, cols=4, permute=False):
    """Print expressions for integers from 0 up to limit or the first unmakeable integer."""
    table = expressions(numbers, ops, nesting, permute)
    print(f'Can make 0 to {unmakeable(table)-1} with expressions({numbers}, ops="{ops}", permute={permute}).'
          f' [{len(table):,} table entries]\n')
    N = limit or unmakeable(table)
    R = ceil(N / cols) # number of rows
    for r in range(R):
        print(*[f'{i:4} = {unbracket(table[i]):<19}'
                for i in range(r, min(N, r + cols * R), R)])
        
def unmakeable(table) -> int:
    """The integer i such that table makes every integer from 0 to i - 1, but not i."""
    for i in range(len(table) + 1):
        if i not in table:
            return i
              
def unbracket(exp: str) -> str:
    "Strip outer parens from exp, if they are there"
    return (exp[1:-1] if exp.startswith('(') and exp.endswith(')') else exp)

# Four 4s with New Mathematical Operations

In [21]:
%time show((4, 4, 4, 4))

Can make 0 to 72 with expressions((4, 4, 4, 4), ops="+-*/^_√!.,", permute=False). [711,642 table entries]

   0 = 44-44                 19 = 4!-(4+(4/4))          38 = 44-(4!/4)             57 = ((.4+4!)/.4)-4     
   1 = 44/44                 20 = 4*(4+(4/4))           39 = 44-(√4/.4)            58 = ((4^4)-4!)/4       
   2 = 4*(4/(4+4))           21 = (4+4.4)/.4            40 = 44-√(4*4)             59 = (4!/.4)-(4/4)      
   3 = (4+(4+4))/4           22 = √4*(44/4)             41 = (.4+(4*4))/.4         60 = 44+(4*4)           
   4 = 4+(4*(4-4))           23 = ((4*4!)-4)/4          42 = √4+(44-4)             61 = (4/4)+(4!/.4)      
   5 = (4+(4*4))/4           24 = 4+(4+(4*4))           43 = 44-(4/4)              62 = (4*(4*4))-√4       
   6 = 4+((4+4)/4)           25 = (4+(4*4!))/4          44 = 4+(44-4)              63 = ((4^4)-4)/4        
   7 = (44/4)-4              26 = 4+(44/√4)             45 = 44+(4/4)              64 = (4+4)*(4+4)        
   8 = 4+(4+(4-4))           

We can also solve the "2016 with four fours" puzzle:

In [22]:
expressions((4, 4, 4, 4), OPS, 2)[2016]

'((4+4)!/(4!-4))'

In a [separate video](https://www.youtube.com/embed/Noo4lN-vSvw), Alex Bellos shows  how to form **every** integer from 0 to infinity using four 4s, if unlimited numbers of square root and `log` functions are allowed. The solution comes from Paul Dirac (although [Dirac originally developed it](https://nebusresearch.wordpress.com/2014/04/18/how-dirac-made-every-number/) for the "four 2s" problem).

Donald Knuth [conjectured](https://www.tandfonline.com/doi/abs/10.1080/0025570X.1964.11975546) that with floor, square root and factorial, you can make any positive integer with just **one** 4. (We would need more efficient ways of dealing with very large numbers to make progress on Knuth's problem.)

Below are some popular variant problems:

# Four 2s

In [23]:
show((2, 2, 2, 2))

Can make 0 to 30 with expressions((2, 2, 2, 2), ops="+-*/^_√!.,", permute=False). [109,291 table entries]

   0 = 22-22                  8 = 2+(2+(2+2))           16 = 2*(2*(2+2))           24 = 22+√(2+2)          
   1 = 22/22                  9 = (22/2)-2              17 = 22-(√.2^-2)           25 = (2-2.2)^-2         
   2 = 2+(2*(2-2))           10 = 22/2.2                18 = 22-(2+2)              26 = 2+(2+22)           
   3 = (2+(2+2))/2           11 = 22/√(2+2)             19 = (2+(2-.2))/.2         27 = 22+(√.2^-2)        
   4 = .2*(22-2)             12 = (2+22)/2              20 = 22-√(2+2)             28 = 2+(2+(2+2)!)       
   5 = 2+(2+(2/2))           13 = 2+(22/2)              21 = 22-(2/2)              29 = 2+(2+(.2^-2))      
   6 = (2*(2+2))-2           14 = (2^(2+2))-2           22 = 2+(22-2)              30 = (2+(2+2))/.2       
   7 = 2+(2/(2*.2))          15 = (2+(2/2))/.2          23 = 22+(2/2)           


# Four 9s

In [24]:
show((9, 9, 9, 9))

Can make 0 to 61 with expressions((9, 9, 9, 9), ops="+-*/^_√!.,", permute=False). [774,333 table entries]

   0 = 99-99                 16 = 9+((9/.9)-√9)         32 = (99-√9)/√9            48 = √9!*(9-(9/9))      
   1 = 99/99                 17 = 9+(9-(9/9))           33 = √9*(99/9)             49 = 9+(√9!*(√9!/.9))   
   2 = (99/9)-9              18 = 99-(9*9)              34 = (√9+99)/√9            50 = ((9*√9!)-9)/.9     
   3 = (9+(9+9))/9           19 = 9+(9+(9/9))           35 = (√9!+99)/√9           51 = 9*(9-(√9/.9))      
   4 = 9-(9/(.9+.9))         20 = 9+(99/9)              36 = 9+(9+(9+9))           52 = √9!*(9-(√9/9))     
   5 = √9+((9+9)/9)          21 = (9+9.9)/.9            37 = (9/.9)+(9*√9)         53 = (9*√9!)-(9/9)      
   6 = ((9+(9+9))/9)!        22 = 9+(√9+(9/.9))         38 = √9!*(√9+(√9/.9))      54 = (9*9)-(9*√9)       
   7 = 9-((9+9)/9)           23 = √9+((9+9)/.9)         39 = 9+(9*(√9/.9))         55 = 99/(.9+.9)         
   8 = ((9*9)-9)/9           

# Four 5s

In [25]:
%time show((5, 5, 5, 5))

Can make 0 to 38 with expressions((5, 5, 5, 5), ops="+-*/^_√!.,", permute=False). [344,933 table entries]

   0 = 55-55                 10 = 55/5.5                20 = 5+(5+(5+5))           30 = 55-(5*5)           
   1 = 55/55                 11 = 5.5+5.5               21 = (5+5.5)/.5            31 = 55-(5!/5)          
   2 = 5!/(5+55)             12 = (5+55)/5              22 = 55/(5*.5)             32 = (5.5-5)^-5         
   3 = (5+(5+5))/5           13 = (5!-55)/5             23 = 55-(.5^-5)            33 = .5*(5!*.55)        
   4 = ((5*5)-5)/5           14 = (5!/5)-(5+5)          24 = (5*5)-(5/5)           34 = 5+(5+(5!/5))       
   5 = 5+(5*(5-5))           15 = (5*5)-(5+5)           25 = .5*(55-5)             35 = 5+(5+(5*5))        
   6 = (55/5)-5              16 = 5+(55/5)              26 = (5/5)+(5*5)           36 = (5!+(.5*5!))/5     
   7 = 5+((5+5)/5)           17 = 5+(5!/(5+5))          27 = (5*5.5)-.5            37 = 5+(.5^-√(5*5))     
   8 = 5.5+(5*.5)            


# Countdown to New Year's

Now another type of New Year's countdown puzzle. On December 31 2017, [Michael Littman](http://cs.brown.edu/~mlittman/) posted this:

> 2+0+1×8, 2+0-1+8, (2+0-1)×8, |2-0-1-8|, -2-0+1×8, -(2+0+1-8), √|2+0-18|, 2+0+1^8, 20-18, 2^(0×18), 2×0×1×8... Happy New Year!

Can we replicate that countdown? For 2018 and for following years?

In [26]:
def countdown(years):
    """Print a countdown using the digits of this year, in order."""
    for year in years:
        table = expressions(tuple(int(d) for d in str(year)))
        for i in reversed(range(11)):
            print(f'{i:2d} = {table[i]}')
        print('\n')
        
countdown(range(2018, 2025))

10 = (2.0+(1*8))
 9 = ((2.0-1)+8)
 8 = ((2.0-1)*8)
 7 = (-2.0+(1+8))
 6 = (-2.0+(1*8))
 5 = (-(2.0+1)+8)
 4 = (20*(1-.8))
 3 = (2.0+(1^8))
 2 = (20-18)
 1 = (2^(.0*18))
 0 = (2*(.0*18))


10 = (20-(1+9))
 9 = ((2.0-1)*9)
 8 = (-2.0+(1+9))
 7 = (-2.0+(1*9))
 6 = (-(2.0+1)+9)
 5 = (.20^(-1^9))
 4 = ((-.20^-1)+9)
 3 = (2.0+(1^9))
 2 = (20/(1+9))
 1 = (20-19)
 0 = (2*(.0*19))


10 = (2/0.20)
 9 = ((20/2)-.0!)
 8 = (2^(.0!+2.0))
 7 = (2+(.0!/.20))
 6 = (2*(.0!+2.0))
 5 = ((2^.0)/.20)
 4 = (20*.20)
 3 = (2+(.02^.0))
 2 = (2+(.0*20))
 1 = (20/20)
 0 = (20-20)


10 = (20/(2*1))
 9 = ((20/2)-1)
 8 = (2.0^(2+1))
 7 = (2+(0.2^-1))
 6 = (2.0*(2+1))
 5 = (2.0+(2+1))
 4 = (20*(2*.1))
 3 = (2.0+(2-1))
 2 = (2+(.0*21))
 1 = (-20+21)
 0 = (2*(.0*21))


10 = (20-(2/.2))
 9 = ((20-2)/2)
 8 = ((20/2)-2)
 7 = (2+(.0!+(2+2)))
 6 = (2.0+(2+2))
 5 = (20/(2+2))
 4 = (-20+(2+2)!)
 3 = (2.0+(2/2))
 2 = (-20+22)
 1 = (20^(2-2))
 0 = (20*(2-2))


10 = (2.0*(2+3))
 9 = (((2^.0)+2)*3)
 8 = (2.0+(2*3))
 7 = ((20/2)-3

# Can you make 24?

In the [538 Riddler for July 10th 2020](https://fivethirtyeight.com/features/can-you-make-24/), Zach Wissner-Gross asks "Can you make 24?" from the digits (2, 3, 3, 4), in any order, with just the five binary operators. 

In [27]:
expressions((2, 3, 3, 4), '+-*/^', permute=True)[24]

'(((3^2)-3)*4)'

For extra credit, Zach asks for **multiple ways** to make 24. 

We haven't dealt with reporting multiple ways to make a number; the `ExpTable` only keeps one. I'll try collecting one expression from each permutation of the numbers. If there are multiple ways from a single permutation we won't keep more than one of those, but this approach won't give you every way (in general), but it should be good enough to answer Zach's question.

In [28]:
def can_make(total: int, numbers: tuple, ops='+-*/^') -> Set[Exp]:
    """Can we make the total from the numbers (in any order)? Return a set of expressions."""
    return {expressions(nums, ops)[total]
            for nums in set(permutations(numbers))
            if total in expressions(nums, ops)}

can_make(24, (2, 3, 3, 4))

{'(((3^2)-3)*4)',
 '(((4-2)^3)*3)',
 '(3*((4-2)^3))',
 '(3*(4^(3/2)))',
 '(3/((2/4)^3))',
 '(4*((3^2)-3))'}

Readers suggested other interesting tuples of numbers:

In [29]:
can_make(24, (2, 3, 8, 8))

{'(((2*8)-8)*3)',
 '(((8*2)-8)*3)',
 '((2^(8-3))-8)',
 '((2^3)+(8+8))',
 '((8-(2+3))*8)',
 '((8-(3+2))*8)',
 '(3*((2*8)-8))',
 '(3*((8*2)-8))',
 '(3*((8+8)/2))',
 '(8*(8-(2+3)))',
 '(8*(8-(3+2)))'}

In [30]:
can_make(24, (2, 3, 10, 10))

{'(((10-3)*2)+10)',
 '((2*(10-3))+10)',
 '((2^10)-(10^3))',
 '(10+((10-3)*2))',
 '(10+(2*(10-3)))',
 '(10-((3-10)*2))',
 '(10-(2*(3-10)))'}

In [31]:
can_make(24, (3, 3, 8, 8))

{'(8/(3-(8/3)))'}

In [32]:
can_make(24, (2, 3, 6, 6))

{'(((3*6)-6)*2)',
 '(((3+2)*6)-6)',
 '(((3+6)*2)+6)',
 '(((6+3)*2)+6)',
 '((2*(3+6))+6)',
 '((2*(6+3))+6)',
 '((2+6)*(6-3))',
 '(6*(6*(2/3)))',
 '(6+((3+6)*2))',
 '(6+((6+3)*2))',
 '(6+(2*(3+6)))',
 '(6+(2*(6+3)))'}

In [33]:
can_make(24, (0, 0, 2, 5))

{'((5^2)-(0^0))'}

This relies on 0<sup>0</sup> = 1, which Python agrees with, but some mathematicians treat as [undefined](https://en.wikipedia.org/wiki/Zero_to_the_power_of_zero).

Nicolas Schank asked for the following in the Facebook "omg math" group:

In [34]:
can_make(24, (13, 11, 10, 6), '+-*/^!')

{'(10-(6/(13-11))!)!'}

In [35]:
can_make(24, (9, 8, 1, 1), '+-*/^!')

{'(8/((1^9)+1))!', '(8/(1+(1^9)))!'}

In [36]:
can_make(24, (9, 7, 7, 7), '+-*/^!')

{'(9!/(7!+(7!+7!)))'}

# Making 6 from 3 Digits

Another Facebook "omg math" problem:

 *For each digit, find a way to express 6 using only that digit exactly three times and arithmetic operations. E.g., for the digit 2, `'2+2+2'` = 6.*

This is easy if "arithmetic operations" include square root and factorial:

In [37]:
for n in range(10):
    print(f"6 = {expressions((n, n, n), '+-*/√!').get(6)}")

6 = (0!+(0!+0!))!
6 = (1+(1+1))!
6 = (2+(2+2))
6 = ((3*3)-3)
6 = (4-(4/4))!
6 = (5+(5/5))
6 = (6+(6-6))
6 = (7-(7/7))
6 = √(8+(8/8))!
6 = ((9+9)/√9)


# Even More Fours: The Power of Floor and Ceiling

With the standard set of `OPS`, we got all the integers up to 72 with four 4s. If we add the floor and ceiling operators, we get a big jump: suddenly, all the results of a division or a square root that didn't form an integer can now be coerced into integers. Instead of getting the integers only up to 72, we now get all the way up to 1644 (although it takes a full minute to get there, and I'll just show the first 100, to sace space):

In [38]:
%time show((4, 4, 4, 4), limit=100, ops=OPS + "⌊⌈")

Can make 0 to 1644 with expressions((4, 4, 4, 4), ops="+-*/^_√!.,⌊⌈", permute=False). [1,184,901 table entries]

   0 = 44-44                 25 = (4+(4*4!))/4          50 = 44+(4!/4)             75 = (4!+(4!/4))/.4     
   1 = 44/44                 26 = 4+(44/√4)             51 = 44+⌈√44⌉              76 = (4*(4!-4))-4       
   2 = 4*(4/(4+4))           27 = 4+(4!-(4/4))          52 = 4+(4+44)              77 = ⌈(44*(4^.4))⌉      
   3 = (4+(4+4))/4           28 = 44-(4*4)              53 = 44+⌊(.4*4!)⌋          78 = (4*(4!-4))-√4      
   4 = ⌊4.444⌋               29 = 4+(4!+(4/4))          54 = 44+(4/.4)             79 = ⌈(4*(4!-4.4))⌉     
   5 = ⌈4.444⌉               30 = (4+(4+4))/.4          55 = 44/(.4+.4)            80 = 4*(4+(4*4))        
   6 = 4+((4+4)/4)           31 = 4!+((4+4!)/4)         56 = 4*(4+(4/.4))          81 = (4-(4/4))^4        
   7 = (44/4)-4              32 = (4*4)+(4*4)           57 = ((.4+4!)/.4)-4        82 = √4+(4*(4!-4))      
   8 = 4+(4+(4-4))     

We could get up to 2892 with `show((4, 4, 4, 4), ops=OPS + "⌊⌈", nesting=3)`.

# Even More Fives: Five 5s

In the [xkcd forum](http://forums.xkcd.com/viewtopic.php?f=14&t=116813&start=280) they took up the problem of **five 5s** and got all the integers up to 298, using the "[double factorial function](https://en.wikipedia.org/wiki/Double_factorial)" and [π function](https://en.wikipedia.org/wiki/Prime-counting_function). We can get up to 171, using just the default operators, but with five digits it does take about seven minutes, whereas all the other puzzles with four or three digits (and without floor and ceiling) took less than a minute. I suspect you could go much further using floor and ceiling, but that computation would take even longer, so for now let's stick with our default set of operations:

In [39]:
%time show((5, 5, 5, 5, 5))

Can make 0 to 171 with expressions((5, 5, 5, 5, 5), ops="+-*/^_√!.,", permute=False). [14,809,921 table entries]

   0 = 5*(55-55)             43 = 55-(5!/(5+5))         86 = (55/.5)-(5!/5)       129 = (5!-55.5)/.5       
   1 = 5^(55-55)             44 = 55-(55/5)             87 = (555-5!)/5           130 = 5!+(55/5.5)        
   2 = 55/(5*5.5)            45 = (5*5!)-555            88 = 5*(.55/(.5^5))       131 = (55+(5*5!))/5      
   3 = .5*((55/5)-5)         46 = 55+((.5-5)/.5)        89 = 5!+((5!/5)-55)       132 = 5!+((5+55)/5)      
   4 = 5-(55/55)             47 = 5+(5+(5+(.5^-5)))     90 = (55-(5+5))/.5        133 = (5+(5!*5.5))/5     
   5 = 5+(55-55)             48 = 5!/(5*(5.5-5))        91 = (5*5)+(5!*.55)       134 = (5!/5)+(55/.5)     
   6 = 5+(55/55)             49 = 55-(5+(5/5))          92 = 5+(55+(.5^-5))       135 = (5!+555)/5         
   7 = ((5+55)/5)-5          50 = 55.5-5.5              93 = .5+(5!-(5*5.5))      136 = 5+(5!+(55/5))      
   8 = .5*(5+(55/5))  

# f(n): Unmakeable from (1, ..., *n*)

The facebook group "actually good math problems" posed the problem of determining the values of `f(n)`, which is defined to be the smallest positive integer that can not be made from, in our terms,  `expressions(range(n + 1), '+-*/^,', permute=True)`. Computing up to `f(5)` is fast, but `f(6)` requires hundreds of billions of combinations and 100 minutes to run; it would take some work to make it more efficient, or to move on to `f(7)`.

In [40]:
@cache
def f(n, ops='+-*/^,') -> int:
    """The smallest integer inexpressible in digits 0 thru n."""
    return unmakeable(expressions(range(n + 1), ops, permute=True))

In [41]:
%%time
for n in range(7):
    print(f'f({n}) = {f(n)}')

f(0) = 1
f(1) = 2
f(2) = 4
f(3) = 25
f(4) = 175
f(5) = 1099
f(6) = 9562
CPU times: user 1h 38min 47s, sys: 4.47 s, total: 1h 38min 52s
Wall time: 1h 42min 19s


# What's Next?

One exercise would be adding even more operators, such as:

- **Nth root**: `3√8` = 2
- **Percent**: `50%` = 0.5
- **Absolute value**: `|1-3|` = 2 (redundant if you have unary minus, but add it if you like it)
- **Repeating decimal**: `.4...` = .44444444... = 4/9
- **Booleans**: `1+(2<3)` = 2, because `(2<3)` is True, which is treated as 1.
- **Double factorial**: `9!!` = 9 × 7 × 5 × 3 × 1 = 945; not the same as `(9!)!`
- **Gamma function**: `Γ(n)` = (n − 1)! and works for non-integers
- **Prime counting function**: `π(n)` = number of primes ≲ n; e.g. `π(5)` = 3
- **Transcendental functions**: `log`, `sin` `cos`, `tan`, `arcsin`, ...  
- **Degree symbol**: `180°` = π
- **Log to base**: `log_10(100)` = 2
- **Matrix notation**: with determinant symbol to get a number.
- **Combinations and Permutations**: `6 C 2` = 15; `6 P 2` = 30

Another approach would be to look around for related puzzles and solve them. What do you want to do?